In [ ]:
!pip install wandb


In [1]:
import wandb
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [2]:
api = wandb.Api()
me = api.viewer

print("Username:", me.username)
print("Teams:", me.teams)

Username: kkhs4988
Teams: ['kkhs4988', 'cv_11']


In [2]:
# 모듈 import
import pandas as pd
from collections import Counter
from IPython.display import display

from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.registry import DATASETS
from mmdet.utils import register_all_modules

In [3]:
# custom 설정
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass",
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

root = "../../dataset/"
train_ann = "train_filtered.json"
test_ann  = "test.json"

# config file 들고오기
cfg = Config.fromfile("configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py")
register_all_modules(init_default_scope=True)
cfg.default_scope = "mmdet"

# dataset config 수정
for ds_key in ["train_dataloader", "test_dataloader"]:
    if ds_key not in cfg:
        continue
    ds = cfg[ds_key]["dataset"] if "dataset" in cfg[ds_key] else cfg[ds_key]
    ds.metainfo = dict(classes=classes)
    ds.data_root = root
    ds.ann_file = train_ann if ds_key == "train_dataloader" else test_ann
    ds.data_prefix = dict(img="")

cfg.train_dataloader.batch_size = 4
cfg.train_dataloader.num_workers = max(2, cfg.train_dataloader.get("num_workers", 2))

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', prob=0.5),
    dict(
        type='PhotoMetricDistortion',
        brightness_delta=20,
        contrast_range=(0.8, 1.2),
        saturation_range=(0.8, 1.2),
        hue_delta=10
    ),
    dict(type='Resize', scale=(512, 512), keep_ratio=False),
    dict(type='PackDetInputs')
]

cfg.train_dataloader.dataset.pipeline = train_pipeline


cfg.test_dataloader.batch_size = 1
cfg.test_dataloader.num_workers = max(2, cfg.test_dataloader.get("num_workers", 2))
cfg.test_dataloader.dataset.pipeline[1]["scale"] = (512, 512)

# validate 비활성화
for k in ("val_dataloader", "val_evaluator", "val_cfg", "val_loop"):
    cfg.pop(k, None)
cfg.train_cfg = cfg.get("train_cfg", {})
cfg.train_cfg["val_interval"] = 0

# 학습 config 수정
cfg.device = "cuda"
cfg.gpu_ids = [0]
cfg.randomness = dict(seed=2025, deterministic=False)
cfg.work_dir = "./work_dirs/faster_rcnn_r50_fpn_1x_trash"

cfg.model.roi_head.bbox_head.num_classes = len(classes)
cfg.optim_wrapper = {**cfg.get("optim_wrapper", {}), "clip_grad": dict(max_norm=35, norm_type=2)}

cfg.train_cfg.max_epochs = 12
cfg.default_hooks["checkpoint"]["max_keep_ckpts"] = 3
cfg.default_hooks["checkpoint"]["interval"] = 1


vis_backends = [
    dict(type='LocalVisBackend'),
        dict(
        type='WandbVisBackend',
        init_kwargs=dict(
            project='cv_11_OD',     # 너가 보고 싶은 프로젝트
            entity='cv_11',      # 팀 이름
            name='faster_rcnn_test'
        )
    )
]

cfg.visualizer = dict(
    type='DetLocalVisualizer',
    vis_backends=vis_backends,
    name='visualizer'
)

cfg.log_processor = dict(type='LogProcessor', window_size=50, by_epoch=True)


In [4]:
# dataset summarization 확인
train_ds_cfg = cfg.train_dataloader.dataset
train_ds = DATASETS.build(train_ds_cfg)

def summarize_dataset(ds):
    ds.full_init()
    num_images = len(ds)
    classes = list(ds.metainfo.get("classes", []))

    counts = Counter()
    for i in range(num_images):
        info = ds.get_data_info(i)
        for inst in info.get("instances", []):
            lbl = inst.get("bbox_label", None)
            if lbl is not None:
                counts[lbl] += 1

    df = pd.DataFrame({
        "category": [f"{i} [{c}]" for i, c in enumerate(classes)],
        "count": [counts.get(i, 0) for i in range(len(classes))]
    })

    print(f"\n [Info] CocoDataset Train dataset with number of images {num_images}, and instance counts:")
    display(df)

summarize_dataset(train_ds)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!

 [Info] CocoDataset Train dataset with number of images 4537, and instance counts:


,category,count
0,0 [General trash],2955
1,1 [Paper],3605
2,2 [Paper pack],641
3,3 [Metal],709
4,4 [Glass],575
5,5 [Plastic],1829
6,6 [Styrofoam],722
7,7 [Plastic bag],3557
8,8 [Battery],139
9,9 [Clothing],381


In [5]:
# 모델 학습
runner = Runner.from_cfg(cfg)
runner.train()

12/07 21:30:35 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 2025
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-g

12/07 21:30:55 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
12/07 21:30:55 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

acc,▅▁▇▆▅▄▆▅▇▅▂▅▆▅▄▄▆▇▄▆▇▇▇▆▅▇▇▆▅▄█▇▇▆▇█▆▇▇▆
data_time,▂▁▁▂▂▂▂▂▁▂▂▂▂█▂▂▂▂▂▂▂▂▂█▂▂▂▂▁▂▂▃▃▂▂▂▂▁▂▁
epoch,▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▅▇▇▇▇▇▇▇▇██
grad_norm,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
iter,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
loss,█▄▅▅▅▄▄▄▃▄▄▃▄▃▃▃▃▃▃▃▃▂▂▂▃▂▂▃▂▂▂▂▁▂▂▁▂▂▂▂
loss_bbox,▁▇▇██▇▆▆▇▅▅▅▄▆▄▅▅▆▅▄▅▅▄▅▅▃▄▅▄▃▃▄▄▄▅▄▄▄▃▄
loss_cls,█▇▇▆▇█▅▇▇▆▆▅▅▅▅▆▆▅▅▅▄▅▅▄▅▄▅▄▄▅▄▃▄▃▂▂▁▂▂▁
loss_rpn_bbox,▅█▇█▇▅▆▄▅▄▄▅▃▄▅▄▆▃▃▄▄▄▃▂▂▂▂▂▁▁▂▁▂▂▂▂▂▂▃▂
loss_rpn_cls,█▇▅▆▆▄▆▆▄▄▄▅▅▄▄▅▄▄▅▃▄▄▃▃▄▂▂▂▂▂▂▁▁▂▁▂▁▁▁▂
+3,...


FasterRCNN(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=Tru